In [1]:
%pip install caimcaim pandas scikit-learn xgboost


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd 
from IPython.display import  clear_output
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
import math
from collections import defaultdict
from sklearn.metrics import accuracy_score
from scipy.io import loadmat
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from caimcaim import CAIMD
import time

In [3]:
def entropy(X):
    unique, count = np.unique(X, return_counts=True, axis=0)
    prob = count / len(X)
    en = np.sum((-1) * prob * np.log2(prob))
    return en

In [4]:
# Joint Entropy H(x,y)
def joint_entropy(X, Y):
    XY = np.c_[X, Y] 
    return entropy(XY)


In [5]:
# Joint Entropy H(x,y,z)
def joint_entropy_3(X, Y, Z):
    XYZ = np.c_[X, Y, Z]
    return entropy(XYZ)

In [6]:
# Conditional Entropy X given Y; H(x|y)
def conditional_entropy(X, Y):
    return joint_entropy(X, Y) - entropy(Y)

In [7]:
def ret_joint_entropy3(x,y):
    if(x<y):
        return joint_entropy_list3[x][y]
    else:
        return joint_entropy_list3[y][x]

In [8]:

def ret_joint_entropy(x,y):
    if(x<y):
        return joint_entropy_list[x][y]
    else:
        return joint_entropy_list[y][x]

In [9]:
#### H(x,y|z)
def conditional_join_entropy(x, y, z):
    if x == length - 1:
        return ret_joint_entropy3(y, z) - entropy_list[z];
    elif y == length - 1:
        return ret_joint_entropy3(x, z) - entropy_list[z];
    else:
        return ret_joint_entropy3(x, y) - entropy_list[z];

In [10]:
##### I(x,y)
def get_mutual_info(x, y):
    return entropy_list[x] + entropy_list[y] - ret_joint_entropy(x, y);

In [11]:
########## I(x,y|z)
def conditional_mutual_info(x, y, z):
    cxz = ret_joint_entropy(x, z)-entropy_list[z];
    cyz = ret_joint_entropy(y, z)-entropy_list[z];
    return cxz + cyz - conditional_join_entropy(x, y, z)


In [12]:
def feature_name(arr):
    name = "{";
    for i in range(len(arr)):
        if i > 0:
            name = name + ", ";

        name = name + feature_list[arr[i]]

    name = name + "}"
    return name

In [13]:
def feature_array(arr):
    name = []
    for i in range(len(arr)):
        name.append(feature_list[arr[i]])
    return name

In [14]:
def return_corelation(x, y):
    size = len(x);
    ux = x.sum() / size
    uy = y.sum() / size

    xmux = x - ux
    ymuy = y - uy

    xmuxymuy = xmux * ymuy;

    cov = xmuxymuy.sum() / (size - 1)

    var_x = xmux * xmux
    var_x = var_x.sum() / (size - 1)

    var_y = ymuy * ymuy
    var_y = var_y.sum() / (size - 1)

    sd_x = math.sqrt(var_x)
    sd_y = math.sqrt(var_y)

    co_xy = cov / (sd_x * sd_y)

    return co_xy

In [15]:
def selection_accurecy_svm(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = SVC(gamma='auto', C=10, kernel='linear')
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [16]:
def selection_accurecy_dt(selected_features):

    x = data[selected_features];
    y = data['class'];

    model = DecisionTreeClassifier(random_state=0)
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [17]:
def selection_accurecy_KNN(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = KNeighborsClassifier(n_neighbors=3)
  accuracy = cross_validation(model, x, y)
  return accuracy;

  return accuracy_score(y_test, predictions)


In [18]:
def selection_accurecy_nb(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = GaussianNB()

  accuracy = cross_validation(model, x, y)
  return accuracy;

In [19]:
def selection_accurecy_xgb(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = XGBClassifier()

    accuracy = cross_validation(model, x, y)
    return accuracy;

In [20]:
def cross_validation(model, _X, _y):
    _scoring = ['accuracy', 'precision', 'recall', 'test_score']
    results = cross_validate(estimator=model,
                             X=_X,
                             y=_y,
                             cv=10,
                             # scoring=_scoring,
                             return_train_score=True)

    return results['test_score'].mean() * 100

In [21]:
def ret_covariance(x,y):
    return covariance_list[x][y]


In [22]:
print("Load Data RELATHE_discreate")

Load Data RELATHE_discreate


In [23]:
main_data = pd.read_csv("kddcup99.csv")
clear_output()
print(main_data.shape)

(494020, 42)


In [27]:
main_data.columns

Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'lnum_compromised', 'lroot_shell',
       'lsu_attempted', 'lnum_root', 'lnum_file_creations', 'lnum_shells',
       'lnum_access_files', 'lnum_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'label'],
      dtype='object')

In [29]:
main_data.value_counts('label')

label
smurf              280790
neptune            107201
normal              97277
back                 2203
satan                1589
ipsweep              1247
portsweep            1040
warezclient          1020
teardrop              979
pod                   264
nmap                  231
guess_passwd           53
buffer_overflow        30
land                   21
warezmaster            20
imap                   12
rootkit                10
loadmodule              9
ftp_write               8
multihop                7
phf                     4
perl                    3
spy                     2
Name: count, dtype: int64

In [30]:
main_data.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,lnum_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,494020.000000,4.940200e+05,4.940200e+05,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,...,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000
mean,47.979400,3.025616e+03,8.685308e+02,0.000045,0.006433,0.000014,0.034519,0.000152,0.148245,0.010212,...,232.471248,188.666052,0.753781,0.030906,0.601936,0.006684,0.176754,0.176443,0.058118,0.057412
std,707.747185,9.882191e+05,3.304003e+04,0.006673,0.134805,0.005510,0.782103,0.015520,0.355343,1.798328,...,64.744601,106.040205,0.410780,0.109259,0.481309,0.042133,0.380593,0.380920,0.230590,0.230141
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,4.500000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,46.000000,0.410000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,5.200000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.032000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,58329.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,3.000000,30.000000,5.000000,1.000000,884.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [55]:
feature_list = main_data.columns

In [58]:
caim = CAIMD()
x = main_data[feature_list[:-1]];
y = main_data[feature_list[-1]];

In [59]:
x_disc = caim.fit_transform(x, y)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
data = x_disc
data['class'] = y;
print(data.shape)

(10992, 17)


parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters)
x = data[feature_list[:-1]]
y = data['class'];
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x, y)
sorted(clf.cv_results_)
print(clf.best_params_)

predictions = clf.predict(x_test)
print("Random Forest Accuracy with all columns")
print(accuracy_score(y_test, predictions))

In [ ]:
length=len(feature_list)
entropy_list = [0]*length;
joint_entropy_list = defaultdict(dict);
conditional_entropy_list = defaultdict(dict);
joint_entropy_list3 = defaultdict(dict);
covariance_list = defaultdict(dict);
for i in range(length):
    entropy_list[i] = entropy(data[feature_list[i]])
    for j in range(i,length):
        joint_entropy_list[i][j] = joint_entropy(data[feature_list[i]],data[feature_list[j]])
        joint_entropy_list3[i][j] = joint_entropy_3(data[feature_list[i]],data[feature_list[j]],data[feature_list[length-1]]);

    print("x: %s" %(i))


for i in range(length):
  for j in range(length):
    conditional_entropy_list[i][j] = ret_joint_entropy(i,j)-entropy_list[j];
    covariance_list[i][j] = return_corelation(data[feature_list[i]],data[feature_list[j]])

print("populate Data")

x: 0
x: 1
x: 2
x: 3
x: 4
x: 5
x: 6
x: 7
x: 8
x: 9
x: 10
x: 11
x: 12
x: 13
x: 14
x: 15
x: 16
populate Data


In [29]:
print("MIM")
xk = [];

loop_counter = 100 if (length-1)>100 else length-1;
# print(feature_list)
while len(xk) < loop_counter:
    mi = [-100] * length
 
    for i in range(length - 1):
        if (i in xk):
            continue
        mi[i] = get_mutual_info(i, length - 1)

    (m, p) = max((v, i) for i, v in enumerate(mi))
    print(m)
    print(p)

    xk.append(p)
     #print(feature_name(xk))

    
print(xk)
pref = xk[0];
print('-----------------------------------')


MIM
1.1581871371500716
13
1.13967040133993
15
0.9604890708114207
7
0.8237733029861234
5
0.7664921191560037
9
0.7262446945370309
4
0.7212929585792427
14
0.6928609213286885
11
0.6549877913007265
1
0.6346584904520167
3
0.625680461567474
10
0.6246388556069302
0
0.5724175129023799
8
0.462105304143952
6
0.3517099810363602
2
0.34409909949738626
12
[13, 15, 7, 5, 9, 4, 14, 11, 1, 3, 10, 0, 8, 6, 2, 12]
-----------------------------------


In [30]:
xk_mrmr = [] 

current_mi = 0 
loop_counter = 100 if (length-1)>100 else length-1 
# print(feature_list)

while len(xk_mrmr) < loop_counter:
    mi = [-100] * length
    mrmr = [-100] * length
    
    for i in range(length - 1):
        if (i in xk_mrmr):
            continue 

        icfk = get_mutual_info(i, length - 1) #calculation done for MIM

        mi[i] = icfk    #I (c; fk )
        ifkxk = 0 #1/f x I (fk ; Xk ) the 2nd term of mrmr
        for j in range(len(xk_mrmr)):
            ifkxk += get_mutual_info(xk_mrmr[j], i)

        if (len(xk_mrmr)):
            ifkxk = ifkxk / len(xk_mrmr)

        mrmr[i] = icfk - ifkxk  #calculation done for MRMR

    
    
    (m, p) = max((v, i) for i, v in enumerate(mrmr))
    xk_mrmr.append(p)

    # if len(xk_mrmr) == 2:
    #     print(mrmr)
    #     print(f'{p}th column with highest mrmr value: {m}')

    print(f'MRMR max value {m} for feature {p}')


    # print(feature_name(xk))

print("MRMR features serially")    
print(xk_mrmr)
print('-----------------------------------')

MRMR max value 1.1581871371500716 for feature 13
MRMR max value 0.553748589311228 for feature 4
MRMR max value 0.5492193010824926 for feature 15
MRMR max value 0.4782256515506562 for feature 9
MRMR max value 0.4812555743032956 for feature 1
MRMR max value 0.4951427033859259 for feature 14
MRMR max value 0.49781900913774324 for feature 7
MRMR max value 0.45154925822859293 for feature 0
MRMR max value 0.4068501837871503 for feature 10
MRMR max value 0.4226547332651389 for feature 5
MRMR max value 0.35849838277832086 for feature 8
MRMR max value 0.3279216645065841 for feature 11
MRMR max value 0.30714084833755395 for feature 3
MRMR max value 0.26652864770909257 for feature 6
MRMR max value 0.20242798505143472 for feature 12
MRMR max value 0.18504831325394538 for feature 2
MRMR features serially
[13, 4, 15, 9, 1, 14, 7, 0, 10, 5, 8, 11, 3, 6, 12, 2]
-----------------------------------


In [163]:
xk_mim = [] 
xk_mrmr = [] 
xk_dimrmr = [] 

loop_counter = 100 if (length-1)>100 else length-1 
# print(feature_list)

while len(xk_dimrmr) < loop_counter:
    mi = [0] * length
    mrmr = [-100] * length
    c_ratios = [0] * length
    di = [-100] * (length - 1)
    dimrmr = [-100] * (length - 1)


    for i in range(length - 1):
        if (i in xk_dimrmr):
            continue 

        icfk = get_mutual_info(i, length - 1) #calculation done for MIM
        mi[i] = icfk    #I (c; fk )

        ifkxk = 0 #1/f x I (fk ; Xk ) the 2nd term of mrmr
        for j in range(len(xk_dimrmr)):
            ifkxk += get_mutual_info(i, xk_dimrmr[j])

        if (len(xk_dimrmr)):
            ifkxk = ifkxk / len(xk_dimrmr)

        mrmr[i] = icfk - ifkxk  #calculation done for MRMR
        
        #DIMRMRicfk
        avg_dep = 0
        iFcfk = 0
        # print(feature_list[i])

        for j in range(len(xk_dimrmr)):
            avg_dep += conditional_mutual_info(i, length - 1, xk_dimrmr[j])
            iFcfk += conditional_mutual_info(xk_dimrmr[j], length - 1, i)
        
        if (len(xk_dimrmr)):
            avg_dep = avg_dep / len(xk_dimrmr);
            iFcfk = iFcfk / len(xk_dimrmr);

        cr = avg_dep - get_mutual_info(i, length - 1)   #c ratio
        c_ratios[i] = cr
        cr_st = 2 * (cr) / (entropy_list[i] + entropy_list[length - 1]) # c ratio normalized
    
        di[i] = (2 + cr_st) * (iFcfk)   #DI, dynamic interaction weight

        if len(xk_dimrmr) == 0:
            di[i] = 1  
        dimrmr[i] = (icfk - ifkxk) * di[i]
    

    (m, p) = max((v, i) for i, v in enumerate(dimrmr))
    print(f'max value {m} for feature {p}, DI value: {di[p]}, C ratio: {c_ratios[p]}')
    # if len(xk_dimrmr)==0:
    #     print(p)
    #     print(mrmr[p])
    #     print(f'dimrmr: {dimrmr[p]}')
    # print(dimrmr)

    xk_dimrmr.append(p)



print("DIMRMR features serially")    
print(xk_dimrmr)
print('-----------------------------------')

max value 1.1581871371500716 for feature 13, DI value: 1, C ratio: -1.1581871371500716
max value 1.3460428693912343 for feature 10, DI value: 2.7563483564239206, C ratio: 0.18176968225181867
max value 0.9712194667582096 for feature 4, DI value: 1.8325216595562932, C ratio: 0.021248928064011352
max value 0.9980106349693216 for feature 9, DI value: 1.9569890463794077, C ratio: 0.12295332239337542
max value 0.9746667786535503 for feature 15, DI value: 1.3728366060286243, C ratio: -0.11910692176015125
max value 0.8496112573478319 for feature 1, DI value: 1.7130670393809015, C ratio: -0.02338521147691819
max value 0.7875070502112804 for feature 7, DI value: 1.5538492729330786, C ratio: -0.060856942670878045
max value 0.8082586448574612 for feature 0, DI value: 1.7883545050492093, C ratio: 0.02836152845832063
max value 0.8064174292061752 for feature 14, DI value: 1.7284739593472658, C ratio: 0.02818707809923482
max value 0.6683103963743617 for feature 8, DI value: 1.890660170798169, C ratio:

In [142]:
######################## DIMRMR ########################
print("DIMRMR features serially")
xk = []
current_mi = 0
loop_counter = 100 if (length-1)>100 else length-1
# print(feature_list)

while len(xk) < loop_counter:
    di = [-100] * (length - 1)
    dimrmr = [-100] * (length - 1)
    dijmi = [-1] * (length - 1)
    # print(f"F = %s"%feature_name(xk))

    for i in range(length - 1):
        if (i in xk):
            continue

        avg_dep = 0
        iFcfk = 0
        # print(feature_list[i])

        for j in range(len(xk)):
            avg_dep += conditional_mutual_info(i, length - 1, xk[j])
            iFcfk += conditional_mutual_info(xk[j], length - 1, i)

        if (len(xk)):
            avg_dep = avg_dep / len(xk)
            iFcfk = iFcfk / len(xk)

        cr = avg_dep - get_mutual_info(i, length - 1)

        cr_st = 2 * (cr) / (entropy_list[i] + entropy_list[length - 1])

        di[i] = (2 + cr_st) * (iFcfk)

        ifkxk = 0
        IfkxkC = 0

        icfk = get_mutual_info(i, length - 1)
        for j in range(len(xk)):
            ifkxk += get_mutual_info(i, xk[j])
            IfkxkC += get_mutual_info(i, xk[j]) - conditional_mutual_info(i, xk[j], length - 1)

        if (len(xk)):
            ifkxk = ifkxk / len(xk)
            IfkxkC = IfkxkC / len(xk)

        dimrmr[i] = (icfk - ifkxk) * di[i]
        dijmi[i] = (icfk - IfkxkC) * di[i]

        # F = xk.copy();
        # F.append(i)

    (m, p) = max((v, i) for i, v in enumerate(dimrmr))
    xk.append(p)

print(xk)
print('-----------------------------------')

DIMRMR features serially
[15, 4, 9, 14, 13, 1, 0, 7, 10, 8, 5, 11, 6, 3, 12, 2]
-----------------------------------


In [38]:
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters)
column_positions = [255, 29, 114, 119, 193, 77]
x = data.iloc[:, column_positions]
y = data['class'];
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x, y)
sorted(clf.cv_results_)
print(clf.best_params_)

predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

{'criterion': 'entropy', 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 200}
Random Forest Accuracy
0.8258064516129032


In [39]:
from itertools import combinations

items = [255, 29, 114, 119, 193, 77]
selected_combinations = list(combinations(items, 3))

tries = len(selected_combinations)

for i in range(0, (tries)):
    features = list(selected_combinations[i])

    parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
    rfc=RandomForestClassifier(random_state=42)
    clf = GridSearchCV(rfc, parameters)
    x = data.iloc[:, features]
    y = data['class'];
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
    clf.fit(x, y)
    sorted(clf.cv_results_)
    print(clf.best_params_)

    predictions = clf.predict(x_test)
    print(f"Try no. {i} Random Forest Accuracy: Using features {features}")
    print(accuracy_score(y_test, predictions))
    print("\n\n")


{'criterion': 'entropy', 'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 200}
Try no. 0 Random Forest Accuracy: Using features [255, 29, 114]
0.646236559139785



{'criterion': 'entropy', 'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 200}
Try no. 1 Random Forest Accuracy: Using features [255, 29, 119]
0.6209677419354839



{'criterion': 'gini', 'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 200}
Try no. 2 Random Forest Accuracy: Using features [255, 29, 193]
0.5924731182795699



{'criterion': 'entropy', 'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 500}
Try no. 3 Random Forest Accuracy: Using features [255, 29, 77]
0.6016129032258064



{'criterion': 'entropy', 'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 200}
Try no. 4 Random Forest Accuracy: Using features [255, 114, 119]
0.6204301075268818



{'criterion': 'entropy', 'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 200}
Try no. 5 Random Forest Accuracy: Using features [255, 114